# 既存資料からチャットボットを作成する(Retrieval Augumented Generation:RAG)

- 参考
  - https://colab.research.google.com/github/nyanta012/demo/blob/main/sentence_retrieval.ipynb
  - https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html
  - https://note.com/mahlab/n/nb6677d0fc7c2
- Chromadbがすごい
  - https://www.trychroma.com/
  - https://github.com/chroma-core/chroma
- ChatGPTのSurvey論文
  - https://arxiv.org/pdf/2304.01852.pdf
  


In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
import openai
import os
# langchain-openai パッケージからインポート
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import os
import PyPDF2

class Document:
    def __init__(self, text, metadata):
        self.page_content = text
        self.metadata = metadata

dir_path = "loadtext/"

documents = []

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    metadata = {'filename': filename}

    # PDFファイルを読み込む
    if filename.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            documents.append(Document(text, metadata))

    # テキストファイルを読み込む
    elif filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(Document(text, metadata))


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=vectordb.as_retriever())

In [6]:
# 質問を作成するために invoke メソッドを使用
questions = qa.invoke("この文章の中で重要ど思う線形代数に関する内容についてを考える質問を10個考えてください")

# 結果の表示
print(questions['result'])

1. 線形代数において、ベクトルとは何を指しますか？
2. 線形写像とは何ですか？具体的な例を説明してください。
3. ベクトルに対する線形変換とは何ですか？
4. '𝑓(𝒙)'と '𝑔(𝑓𝒙)'の違いについて説明してください。
5. 線形代数の中で、写像の合成とは何を指しますか？
6. '𝐾𝑒𝑟𝑓'とは何を意味しますか？
7. '𝑑𝑖𝑚𝑉=dim𝐾𝑒𝑟𝑓+dim(𝐼𝑚𝑓)'という等式の意味を説明してください。
8. 線形代数における連続した一次方程式の解き方について説明してください。
9. 線形代数を活用したリアルワールドのアプリケーションは何が考えられますか？
10. Pythonで線形代数の問題を解く際の一般的な手法について説明してください。


In [7]:
# 質問を作成するために invoke メソッドを使用
questions = qa.invoke("疑似逆行列とは何ですか？検索できなければわからないと答えてください。")

# 結果の表示
print(questions['result'])

私は疑似逆行列についての具体的な知識を持っていません。検索機能を使用することができないため、セキュリティ上の理由から詳細な説明は提供できません。あなたの質問に対する疑似逆行列の詳細な説明を得るためには、信頼できる資料や参考書籍を調べてみてください。


In [ ]:
# インポート
from langchain.vectorstores import Chroma
from langchain.text_splitter import Rec1ursiveCharacterTextSplitter
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import gradio as gr
import openai
import os

# ドキュメントをロード
dir_path = 'loadtext/'

documents = []

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    metadata = {'filename': filename}

    # PDFファイルを読み込む
    if filename.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            documents.append(Document(text, metadata))

    # テキストファイルを読み込む
    elif filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(Document(text, metadata))


# ドキュメントの分割とベクトルDBの作成
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

# QAチェーンの作成
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=vectordb.as_retriever())

# チャット機能の実装
def chat_with_ai(input_text):
    response = qa.invoke(input_text+'検索できた範囲で答えてください。検索できないものは「わかりません」と答えてください。')
    return response['result']

# Gradioインターフェースの設定
iface = gr.Interface(
    fn=chat_with_ai,
    inputs="text",
    outputs="text",
    title="AI Chat with Gradio",
    description="OpenAIを使ったチャットインターフェース"
)

# インターフェースの起動
iface.launch(share=True)

c:\Users\Takayama Ryuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://281b2fb5562b54719d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
# インポート
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import gradio as gr
import openai
import os

# ドキュメントをロードするクラス
class Document:
    def __init__(self, text, metadata):
        self.page_content = text
        self.metadata = metadata

# ドキュメントをロード（例としてテキストを使用）
dir_path = 'loadtext2'

documents = []

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    metadata = {'filename': filename}

    # PDFファイルを読み込む
    if filename.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            documents.append(Document(text, metadata))

    # テキストファイルを読み込む
    elif filename.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(Document(text, metadata))

# ドキュメントの分割とベクトルDBの作成
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

# QAチェーンの作成
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=vectordb.as_retriever())

# チャット機能の実装
def chat_with_ai(input_text, history):
    response = qa({"query": input_text+'検索できた範囲で答えてください。検索できないものは「わかりません」と答えてください。'})
    return response['result']

# GradioのChatInterfaceを使用してインターフェースを作成
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(fn=chat_with_ai, chatbot=gr.Chatbot())
    gr.Markdown("### AI Chat with Gradio")

# インターフェースの起動
demo.launch(share=True)

/var/folders/nq/n3nkdz2558g2zjbwzth6xy700000gn/T/ipykernel_73588/350495278.py:58: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.ChatInterface(fn=chat_with_ai, chatbot=gr.Chatbot())
c:\Users\Takayama Ryuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://a6aed4215b1e517d29.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
